In [39]:
import numpy as np
from cities.utils.data_grabber import find_repo_root, DataGrabber
import pandas as pd
import numpy as np
import requests

data = DataGrabber()
data.get_features_wide(["gdp"])
gdp = data.wide["gdp"]

root = find_repo_root()

In [31]:
area = 'county:*&in=state:*' 

variables = "S0101_C01_001E,S0101_C01_002E,S0101_C01_003E,S0101_C01_004E,S0101_C01_005E,S0101_C01_006E,S0101_C01_007E,S0101_C01_008E,S0101_C01_009E,S0101_C01_010E,S0101_C01_011E,S0101_C01_012E,S0101_C01_013E,S0101_C01_014E,S0101_C01_015E,S0101_C01_016E,S0101_C01_017E,S0101_C01_018E,S0101_C01_019E"


# interval = list(range(2010, 2023)) # it is possible to create this variable as a time series (only for ACS 5 year estimates)

year = 2021

url = f'https://api.census.gov/data/{year}/acs/acs5/subject?get={variables}&for={area}'

response = requests.get(url)

assert response.status_code == 200 # 200 means success

data = response.json()

df = pd.DataFrame(data[1:], columns=data[0])
df.columns

Index(['S0101_C01_001E', 'S0101_C01_002E', 'S0101_C01_003E', 'S0101_C01_004E',
       'S0101_C01_005E', 'S0101_C01_006E', 'S0101_C01_007E', 'S0101_C01_008E',
       'S0101_C01_009E', 'S0101_C01_010E', 'S0101_C01_011E', 'S0101_C01_012E',
       'S0101_C01_013E', 'S0101_C01_014E', 'S0101_C01_015E', 'S0101_C01_016E',
       'S0101_C01_017E', 'S0101_C01_018E', 'S0101_C01_019E', 'state',
       'county'],
      dtype='object')

In [32]:
df

df_subset = df[df['state'] == '09']
df_subset


    # 09000        Connecticut
    # 09001        Fairfield County
    # 09003        Hartford County
    # 09005        Litchfield County
    # 09007        Middlesex County
    # 09009        New Haven County
    # 09011        New London County
    # 09013        Tolland County
    # 09015        Windham County
    
    
    


,S0101_C01_001E,S0101_C01_002E,S0101_C01_003E,S0101_C01_004E,S0101_C01_005E,S0101_C01_006E,S0101_C01_007E,S0101_C01_008E,S0101_C01_009E,S0101_C01_010E,...,S0101_C01_012E,S0101_C01_013E,S0101_C01_014E,S0101_C01_015E,S0101_C01_016E,S0101_C01_017E,S0101_C01_018E,S0101_C01_019E,state,county
309,956446,51434,57243,65500,67324,59983,53400,55581,58664,62312,...,71989,72231,63852,47436,38220,25801,17278,22650,09,001
310,898636,47594,50497,56376,58581,56904,58657,59717,58702,55263,...,62043,65035,60220,49380,37637,24843,17726,23179,09,003
311,185175,7619,9226,9795,10870,9746,9625,9804,10901,9247,...,14569,15197,17114,12652,10915,6413,3780,5899,09,005
312,164568,6544,7519,8791,10316,10041,9179,9328,9923,8944,...,12487,13857,13188,11008,8724,5771,3978,4292,09,007
313,864751,44144,46404,52030,59040,58621,57823,57022,52902,51349,...,59734,62952,58923,47979,38490,24868,16520,22042,09,009
314,269131,13084,12672,17005,16922,19483,17884,17125,16432,14245,...,18742,21261,18942,15980,13058,8337,5668,6315,09,011
315,150120,6132,6709,8000,14472,19024,8105,8106,8644,7376,...,9748,10415,10556,7828,6397,4392,2712,2603,09,013
316,116503,5571,6270,6874,8265,7568,7042,7463,7597,6597,...,8367,8740,9233,6028,5677,2945,2117,2641,09,015


In [33]:
age = df.copy()

In [34]:
column_mapping = {
    'S0101_C01_001E': 'total_pop',
    'S0101_C01_002E': 'under_5',
    'S0101_C01_003E': '5_to_9',
    'S0101_C01_004E': '10_to_14',
    'S0101_C01_005E': '15_to_19',
    'S0101_C01_006E': '20_to_24',
    'S0101_C01_007E': '25_to_29',
    'S0101_C01_008E': '30_to_34',
    'S0101_C01_009E': '35_to_39',
    'S0101_C01_010E': '40_to_44',
    'S0101_C01_011E': '45_to_49',
    'S0101_C01_012E': '50_to_54',
    'S0101_C01_013E': '55_to_59',
    'S0101_C01_014E': '60_to_64',
    'S0101_C01_015E': '65_to_69',
    'S0101_C01_016E': '70_to_74',
    'S0101_C01_017E': '75_to_79',
    'S0101_C01_018E': '80_to_84',
    'S0101_C01_019E': '85_over'
}

age.rename(columns=column_mapping, inplace=True)


In [35]:
age['GeoFIPS'] = age['state'] + age['county']
age.drop(['state', 'county'], axis=1, inplace=True)
age['GeoFIPS'] = age['GeoFIPS'].astype(np.int64)

age.head()


,total_pop,under_5,5_to_9,10_to_14,15_to_19,20_to_24,25_to_29,30_to_34,35_to_39,40_to_44,45_to_49,50_to_54,55_to_59,60_to_64,65_to_69,70_to_74,75_to_79,80_to_84,85_over,GeoFIPS
0,58239,3318,3716,4004,3939,3648,3706,3721,4236,3515,4129,3875,4164,3453,2627,2591,1873,801,923,1001
1,227131,12035,11730,16407,13801,11181,12288,12652,14701,13001,14719,14816,15999,16996,14740,13930,9004,4977,4154,1003
2,25259,1320,1601,1422,1487,1434,1768,1783,1738,1455,1575,1620,1540,1715,1531,1353,995,497,425,1005
3,22412,1196,1274,1230,1463,1192,1817,1517,1625,1437,1689,1386,1584,1408,1258,886,793,344,313,1007
4,58884,3467,3578,4194,3591,3386,3665,3367,3278,3859,3914,3920,4064,4017,3380,2855,1665,1553,1131,1009


In [36]:
common_fips = np.intersect1d(age['GeoFIPS'].unique(), gdp['GeoFIPS'].unique())
age = age[age['GeoFIPS'].isin(common_fips)]
age = age.merge(gdp[['GeoFIPS', 'GeoName']], on='GeoFIPS', how='left')
age = age.reindex(columns=['GeoFIPS', 'GeoName'] + list(age.columns.drop(['GeoFIPS', 'GeoName'])))
age.iloc[:, 2:] = age.iloc[:, 2:].astype(float)
print(age.head())
print(age.shape[0])

   GeoFIPS      GeoName total_pop  under_5   5_to_9 10_to_14 15_to_19  \
0     1001  Autauga, AL   58239.0   3318.0   3716.0   4004.0   3939.0   
1     1003  Baldwin, AL  227131.0  12035.0  11730.0  16407.0  13801.0   
2     1005  Barbour, AL   25259.0   1320.0   1601.0   1422.0   1487.0   
3     1007     Bibb, AL   22412.0   1196.0   1274.0   1230.0   1463.0   
4     1009   Blount, AL   58884.0   3467.0   3578.0   4194.0   3591.0   

  20_to_24 25_to_29 30_to_34  ... 40_to_44 45_to_49 50_to_54 55_to_59  \
0   3648.0   3706.0   3721.0  ...   3515.0   4129.0   3875.0   4164.0   
1  11181.0  12288.0  12652.0  ...  13001.0  14719.0  14816.0  15999.0   
2   1434.0   1768.0   1783.0  ...   1455.0   1575.0   1620.0   1540.0   
3   1192.0   1817.0   1517.0  ...   1437.0   1689.0   1386.0   1584.0   
4   3386.0   3665.0   3367.0  ...   3859.0   3914.0   3920.0   4064.0   

  60_to_64 65_to_69 70_to_74 75_to_79 80_to_84 85_over  
0   3453.0   2627.0   2591.0   1873.0    801.0   923.0  
1  16996

In [37]:
age.to_csv(f"{root}/data/raw/age.csv", index=False)

In [35]:
age_percentage = age.copy()
age_percentage.iloc[:, 2:] = age_percentage.iloc[:, 2:].div(age_percentage['total_pop'], axis=0) * 100
age_percentage.drop('total_pop', axis=1, inplace=True)
age_percentage.head()


,GeoFIPS,GeoName,under_5,5_to_9,10_to_14,15_to_19,20_to_24,25_to_29,30_to_34,35_to_39,40_to_44,45_to_49,50_to_54,55_to_59,60_to_64,65_to_69,70_to_74,75_to_79,80_to_84,85_over
0,1001,"Autauga, AL",5.735096,5.639795,7.447116,6.677899,6.107793,6.342642,6.277973,6.861694,6.638757,6.926363,6.485594,7.283743,5.959735,4.557445,4.463845,3.616344,1.443134,1.535032
1,1003,"Baldwin, AL",5.185503,5.211636,6.913289,6.134864,5.000428,5.349584,5.676891,6.359352,5.709022,6.286094,6.556422,6.779196,7.631737,6.543141,6.172993,4.191158,2.299717,1.998972
2,1005,"Barbour, AL",5.221691,6.49596,5.378462,5.848776,6.005547,6.793424,6.745186,6.717048,5.748282,5.937211,6.347228,6.262813,6.696949,6.327129,5.209631,4.494111,1.865177,1.905374
3,1007,"Bibb, AL",5.199766,6.462631,5.851422,5.878387,5.037976,7.59067,7.226641,6.148038,7.433374,7.316525,5.905353,6.561503,6.606445,5.833446,4.076221,3.653768,1.734753,1.483079
4,1009,"Blount, AL",5.772128,5.966789,6.99934,6.271476,5.899081,6.229159,5.643482,5.993872,6.076815,6.50507,6.598168,6.99934,6.779288,5.533456,5.122129,3.217834,2.682939,1.709633
